<a href="https://colab.research.google.com/github/sou-06/AlignedFasta_To_Excel/blob/main/Aligned_Fasta_to_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

← Upload **aligned fasta** file from folder 📁 icon. Drag and drop your file.

↓ Parameter setting and input file name. Select color from (https://matplotlib.org/2.0.2/examples/color/named_colors.html)


In [50]:
inputfile_name = "your_input_file.fasta" #Please change here for your uploaded filename.
outputfile = "output.xlsx"

Color_threshold = [0.50, 0.75]
bg_colors = ["lightgray", "black"]
font_colors = ["black", "white"]

Consensus_threshold = [0.50, 0.75] # first for lowercase, second for uppercase.
row_break_length = 120
font = "Courier New"

In [ ]:
!pip install biopython openpyxl

In [30]:
from Bio import SeqIO
import matplotlib
from openpyxl import Workbook
from openpyxl.styles import Alignment, PatternFill, Font
from openpyxl.utils import get_column_letter
from collections import Counter

In [51]:

# Import
ids = []
sequences = []
for record in SeqIO.parse(inputfile_name, "fasta"):
    ids.append(record.id)
    sequences.append(str(record.seq).upper())

# Length adjustment
max_len = max(len(seq) for seq in sequences)
aligned_seqs = [seq.ljust(max_len, "-") for seq in sequences]

# Making Excel
wb = Workbook()
ws = wb.active

# Counta for everygene
pos_counter = [0] * len(ids)

ws.cell(row=1, column=1, value="ID")


count = 0  # Header number
block_idx = 0  # Block number

for block_start in range(0, max_len, row_break_length):

    block_end = min(block_start + row_break_length, max_len)
    block_idx += 1

    # First line of block
    base_row = (block_idx - 1) * (len(ids) + 3) + 1

    # Res_Number
    for row_idx, seq_id in enumerate(ids, start=base_row + 1):
        cell = ws.cell(row = row_idx, column = 1, value = seq_id)
    # consensus
    consensus_row = base_row + len(ids) + 1
    ws.cell(row=consensus_row, column=1, value="Consensus")

    # == Prosess each block ==
    for col_idx in range(block_start, block_end):
        excel_col = col_idx - block_start + 2

        # --- Header ---
        column_label = ""
        if aligned_seqs[0][col_idx] != "-":
            count += 1
            if count % 10 == 0:
                cell = ws.cell(row=base_row, column=excel_col, value = str(count))
                #cell.alignment = Alignment(shrinkToFit=True)

        # --- homology ---
        column_bases = [seq[col_idx] for seq in aligned_seqs]
        bases = [b for b in column_bases if b not in ["-", "*"]]

        if len(bases) == 0:
            homology = 0
            most_common_base = "-"
        else:
            cnt = Counter(bases)
            most_common_base, val = cnt.most_common(1)[0]
            homology = val / len(column_bases)


        for j, seq in enumerate(aligned_seqs):

            cell = ws.cell(row=base_row + j + 1, column=excel_col, value=seq[col_idx])
            cell.font = Font(name=font)
            # Count
            if seq[col_idx] not in ["-", "*"]:
                pos_counter[j] += 1

            # Coloring
            t = len(Color_threshold) - 1
            while t >= 0 and seq[col_idx] == most_common_base:
                if homology >= Color_threshold[t]:
                    fill_color = matplotlib.colors.cnames[bg_colors[t]][1:]
                    font_color = matplotlib.colors.cnames[font_colors[t]][1:]
                    cell.fill = PatternFill("solid", start_color=fill_color, end_color=fill_color)
                    cell.font = Font(color=font_color, name=font)
                    break
                t -= 1

        # --- Consensus ---
        if homology >= Consensus_threshold[1]:
            ws.cell(row=consensus_row, column=excel_col, value=most_common_base.upper())
        elif homology >= Consensus_threshold[0]:
            ws.cell(row=consensus_row, column=excel_col, value=most_common_base.lower())

        # --- Col width ---
        col_letter = get_column_letter(excel_col)
        ws.column_dimensions[col_letter].width = 2

    # --- Regidue number at right side ---
    right_col = row_break_length + 2
    for j in range(len(ids)):
        cell = ws.cell(row=base_row + j + 1, column=right_col, value = pos_counter[j])

    ws.column_dimensions[get_column_letter(right_col)].width = 6

# Change font family
for row in ws.iter_rows():
    for cell in row:
        cell.font = Font(name=font ,color=cell.font.color)

wb.save(outputfile)
print("Excel exported:", outputfile)

Excel exported: output.xlsx
